In [1]:
pip install selenium


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install webdriver-manager



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
state_links=["https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/rajasthan-state-road-transport-corporation",
             "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"]

In [5]:
state_links[4]

'https://www.redbus.in/online-booking/rajasthan-state-road-transport-corporation'

**KERALA STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()
#to go to URL
driver.get(state_links[0])
time.sleep(3) #wait for page to load

# Function to get bus links and routes
def kerala_link_route(path):
    LINKS_KERALA = []  #list for route links
    ROUTE_KERALA = []  #list for route name
    
    # WebDriverWait instance
    wait = WebDriverWait(driver, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver.find_elements(By.XPATH, path)   #here path variable is used for getting arguments and the parameter is fixed ...it returns the element which contains the text and link
            for links in paths:  #one page contains many anchor tags with class = "routes" from that we traverse using 'links' variable 
                d = links.get_attribute("href")  #get_attribute is used to get the value used inside the html attribute...it is where we have retrieved link from .
                if d:
                    LINKS_KERALA.append(d)  #d is the link now...we have appended/added in the LINKS_KERALA list
            
            for route in paths:
                ROUTE_KERALA.append(route.text)  #similarly for each anchor tag class='route', we are retrieving the route name by using route.text where .text is used to get value from the element
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']") 
                active_page_number = active_page_element.text #the pagination has numbers in it...that number is stored by using .text in active_page_number 
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"  #text() function is used to match the content with which we provided in the descendent of the div[class = " DC_117_pagination"]
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))  #we have to wait untill the webdriver loads the nextpage's XPATH
                
                # Scroll into view and click using JavaScript if necessary
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click() #we waited until next page xpath loaded...now after loaded we click
                except ElementNotInteractableException:
                    driver.execute_script("arguments[0].click();", next_page_button) 

                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content  #this runs in loop...each page loads its data in 'try'  and then after finishing another page loads
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found") 
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_KERALA, ROUTE_KERALA  

# Retrieve links and routes
LINKS_KERALA, ROUTE_KERALA = kerala_link_route("//a[@class='route']")  #function calling and argument is passed as XPATH 

# Close the WebDriver
driver.quit()

# Save data to a CSV file
df_k = pd.DataFrame({"Route_name": ROUTE_KERALA, "Route_link": LINKS_KERALA})

df_k.to_csv("kerala_route.csv", index=False)

Navigating to page 2
No more pages to paginate or pagination element not found


In [8]:
df_k

,Route_name,Route_link
0,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
5,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
6,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
7,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Thiruvananthapuram to Kozhikode,https://www.redbus.in/bus-tickets/thiruvananth...
9,Bangalore to Kannur (Kerala),https://www.redbus.in/bus-tickets/bangalore-to...


**ANDHRA STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [9]:
driver_A = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_A.maximize_window()
driver_A.get(state_links[1])
time.sleep(3) 
def andhra_link_route(path):
    LINKS_ANDHRA = []
    ROUTE_ANDHRA = []
    wait_A = WebDriverWait(driver_A, 10)
    
    while True:
        try:
            paths = driver_A.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_ANDHRA.append(d)
            
            for route in paths:
                ROUTE_ANDHRA.append(route.text)
            try:
                active_page_element = driver_A.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1) 
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_A.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver_A.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  
                time.sleep(1) 
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_A.execute_script("arguments[0].click();", next_page_button)  
                print(f"Navigating to page {next_page_number}")
                time.sleep(10) 
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_ANDHRA, ROUTE_ANDHRA

# Retrieve links and routes
LINKS_ANDHRA, ROUTE_ANDHRA = andhra_link_route("//a[@class='route']")

# Close the WebDriver
driver_A.quit()

# Save data to a CSV file
df_a = pd.DataFrame({"Route_name": ROUTE_ANDHRA, "Route_link": LINKS_ANDHRA})
df_a.to_csv("ANDHRA_route.csv", index=False)

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found


In [9]:
df_a

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
3,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...
4,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
5,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...
7,Bangalore to Anantapur (andhra pradesh),https://www.redbus.in/bus-tickets/bangalore-to...
8,Bangalore to Chittoor (Andhra Pradesh),https://www.redbus.in/bus-tickets/bangalore-to...
9,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...


**TELUNGANA STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [10]:
driver_T = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_T.maximize_window()
driver_T.get(state_links[2])
time.sleep(3)
def telungana_link_route(path):
    LINKS_TELUNGANA = []
    ROUTE_TELUNGANA = []
    wait_T = WebDriverWait(driver_T, 10)
    while True:
        try:
            paths = driver_T.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_TELUNGANA.append(d)
            
            for route in paths:
                ROUTE_TELUNGANA.append(route.text)
            try:
                active_page_element = driver_T.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1) 
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_T.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver_T.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  
                time.sleep(1)  
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_T.execute_script("arguments[0].click();", next_page_button) 
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_TELUNGANA, ROUTE_TELUNGANA

# Retrieve links and routes
LINKS_TELUNGANA, ROUTE_TELUNGANA = telungana_link_route("//a[@class='route']")

# Close the WebDriver
driver_T.quit()

# Save data to a CSV file
df_t = pd.DataFrame({"Route_name": ROUTE_TELUNGANA, "Route_link": LINKS_TELUNGANA})
df_t.to_csv("TELUNGANA_route.csv", index=False)

Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found


In [11]:
df_t

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
5,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Sathupally,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...


**GOA STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [12]:
driver_G = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_G.maximize_window()
driver_G.get(state_links[3])
time.sleep(3)
def goa_link_route(path):
    LINKS_GOA = []
    ROUTE_GOA = []    
    wait_G = WebDriverWait(driver_G, 10)    
    while True:
        try:
            paths = driver_G.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_GOA.append(d)
            
            for route in paths:
                ROUTE_GOA.append(route.text)
            
            try:
                active_page_element = driver_G.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_G.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver_G.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_G.execute_script("arguments[0].click();", next_page_button) 
                print(f"Navigating to page {next_page_number}")
                time.sleep(10) 
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_GOA, ROUTE_GOA


LINKS_GOA, ROUTE_GOA = goa_link_route("//a[@class='route']")

driver_G.quit()

df_g = pd.DataFrame({"Route_name": ROUTE_TELUNGANA, "Route_link": LINKS_TELUNGANA})
df_g.to_csv("GOA_route.csv", index=False)

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found


In [13]:
df_g

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
5,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Sathupally,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Bhadrachalam,https://www.redbus.in/bus-tickets/hyderabad-to...


**RAJASTHAN STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [14]:
driver_R = webdriver.Chrome()
driver_R.get(state_links[4])
driver_R.maximize_window()
time.sleep(3)
def rajasthan_link_route(path):
    LINKS_RAJASTHAN = []
    ROUTE_RAJASTHAN = []
    wait_R = WebDriverWait(driver_R, 10)
    while True:
        try:
            paths = driver_R.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_RAJASTHAN.append(d)
            
            for route in paths:
                ROUTE_RAJASTHAN.append(route.text)
            
            try:
                active_page_element = driver_R.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_R.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver_R.execute_script("arguments[0].scrollIntoView(true);", next_page_button) 
                time.sleep(1)  
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_R.execute_script("arguments[0].click();", next_page_button) 
                print(f"Navigating to page {next_page_number}")
                time.sleep(10) 
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_RAJASTHAN, ROUTE_RAJASTHAN

# Retrieve links and routes
LINKS_RAJASTHAN, ROUTE_RAJASTHAN = rajasthan_link_route("//a[@class='route']")

# Close the WebDriver
driver_R.quit()

# Save data to a CSV file
df_r = pd.DataFrame({"Route_name": ROUTE_RAJASTHAN, "Route_link": LINKS_RAJASTHAN})
df_r.to_csv("RAJASTHAN_route.csv", index=False)

Navigating to page 2
No more pages to paginate or pagination element not found


In [15]:
df_r

,Route_name,Route_link
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


**SOUTH BENGAL STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [16]:
driver_S = webdriver.Chrome()
driver_S.get(state_links[5])
driver_S.maximize_window()
time.sleep(10)
def southbengal_link_route(path):
    LINKS_SB = []
    ROUTE_SB = []
    wait_S = WebDriverWait(driver_S, 10)
    while True:
        try:
            paths = driver_S.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_SB.append(d)
            
            for route in paths:
                ROUTE_SB.append(route.text)
            try:
                active_page_element = driver_S.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_S.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver_S.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  
                time.sleep(1)
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_S.execute_script("arguments[0].click();", next_page_button) 
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_SB, ROUTE_SB

# Retrieve links and routes
LINKS_SB, ROUTE_SB = southbengal_link_route("//a[@class='route']")

# Close the WebDriver
driver_S.quit()

# Save data to a CSV file
df_s = pd.DataFrame({"Route_name": ROUTE_SB, "Route_link": LINKS_SB})
df_s.to_csv("south_BENGAL_route.csv", index=False)

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found


In [17]:
df_s

,Route_name,Route_link
0,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
3,Haldia to Kolkata,https://www.redbus.in/bus-tickets/haldia-to-ko...
4,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
5,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
6,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
7,Kolkata to Midnapore,https://www.redbus.in/bus-tickets/kolkata-to-m...
8,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
9,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...


**HARYANA STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [18]:
driver_H = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_H.maximize_window()
driver_H.get(state_links[6])
time.sleep(3)
def haryana_link_route(path):
    LINKS_HY = []
    ROUTE_HY = []
    wait_H = WebDriverWait(driver_H, 10)
    while True:
        try:
            paths = driver_H.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_HY.append(d)
            
            for route in paths:
                ROUTE_HY.append(route.text)
            try:
                active_page_element = driver_H.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1) 
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_H.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver_H.execute_script("arguments[0].scrollIntoView(true);", next_page_button) 
                time.sleep(1) 
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_H.execute_script("arguments[0].click();", next_page_button) 
                print(f"Navigating to page {next_page_number}")
                time.sleep(10) 
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_HY, ROUTE_HY

# Retrieve links and routes
LINKS_HY, ROUTE_HY = haryana_link_route("//a[@class='route']")

# Close the WebDriver
driver_H.quit()

# Save data to a CSV file
df_h = pd.DataFrame({"Route_name": ROUTE_HY, "Route_link": LINKS_HY})
df_h.to_csv("haryana_route.csv", index=False)

Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found


In [19]:
df_h

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
2,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
3,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
4,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
5,Manali to Chandigarh,https://www.redbus.in/bus-tickets/manali-to-ch...
6,Shimla to Chandigarh,https://www.redbus.in/bus-tickets/shimla-to-ch...
7,Chandigarh to Manali,https://www.redbus.in/bus-tickets/chandigarh-t...
8,Shimla to Manali,https://www.redbus.in/bus-tickets/shimla-to-ma...
9,Dharamshala (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/dharamshala-...


**ASSAM STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [20]:
driver_AS = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_AS.maximize_window()
driver_AS.get(state_links[7])
time.sleep(3) 
def assam_link_route(path):
    LINKS_AS = []
    ROUTE_AS = []
    
    wait_AS = WebDriverWait(driver_AS, 10)
    
    while True:
        try:
            paths = driver_AS.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_AS.append(d)
            
            for route in paths:
                ROUTE_AS.append(route.text)
            try:
                active_page_element = driver_AS.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)                 
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_AS.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver_AS.execute_script("arguments[0].scrollIntoView(true);", next_page_button) 
                time.sleep(1)
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_AS.execute_script("arguments[0].click();", next_page_button)  
                print(f"Navigating to page {next_page_number}")
                time.sleep(10) 
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_AS, ROUTE_AS

# Retrieve links and routes
LINKS_AS, ROUTE_AS = assam_link_route("//a[@class='route']")

# Close the WebDriver
driver_AS.quit()

# Save data to a CSV file
df_as = pd.DataFrame({"Route_name": ROUTE_AS, "Route_link": LINKS_AS})
df_as.to_csv("assam_route.csv", index=False)

Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found


In [21]:
df_as

,Route_name,Route_link
0,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
1,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
2,Guwahati to Dhubri,https://www.redbus.in/bus-tickets/guwahati-to-...
3,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
4,North Lakhimpur to Sibsagar (Assam),https://www.redbus.in/bus-tickets/north-lakhim...
5,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
6,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...
7,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
8,Sibsagar (Assam) to North Lakhimpur,https://www.redbus.in/bus-tickets/sibsagar-to-...
9,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...


**UTTAR PRADESH STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [22]:
driver_U = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_U.maximize_window()
driver_U.get(state_links[8])
time.sleep(3) 
def up_link_route(path):
    LINKS_UP = []
    ROUTE_UP = []
    wait_U = WebDriverWait(driver_U, 10)
    while True:
        try:
            paths = driver_U.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_UP.append(d)
            for route in paths:
                ROUTE_UP.append(route.text)
            try:
                active_page_element = driver_U.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_U.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver_U.execute_script("arguments[0].scrollIntoView(true);", next_page_button) 
                time.sleep(1)
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_U.execute_script("arguments[0].click();", next_page_button)  
                print(f"Navigating to page {next_page_number}")
                time.sleep(10) 
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_UP, ROUTE_UP

# Retrieve links and routes
LINKS_UP, ROUTE_UP = up_link_route("//a[@class='route']")

# Close the WebDriver
driver_U.quit()

# Save data to a CSV file
df_up = pd.DataFrame({"Route_name": ROUTE_UP, "Route_link": LINKS_UP})
df_up.to_csv("UTTAR_PRADESH_route.csv", index=False)

Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found


In [23]:
df_up

,Route_name,Route_link
0,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
1,Lucknow to Prayagraj(Uttar Pradesh),https://www.redbus.in/bus-tickets/lucknow-to-a...
2,Prayagraj(Uttar Pradesh) to Lucknow,https://www.redbus.in/bus-tickets/allahabad-to...
3,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
4,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
5,Lucknow to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/lucknow-to-b...
6,Lucknow to Agra,https://www.redbus.in/bus-tickets/lucknow-to-agra
7,Ayodhya to Allahabad,https://www.redbus.in/bus-tickets/ayodhya-to-a...
8,Prayagraj(Uttar Pradesh) to Ayodhya,https://www.redbus.in/bus-tickets/allahabad-to...
9,Bareilly (Uttar Pradesh) to Lucknow,https://www.redbus.in/bus-tickets/bareilly-to-...


**WEST BENGAL STATE LINKS(ROUTE_NAME AND ROUTE_LINKS)**

In [24]:
driver_W = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_W.maximize_window()
driver_W.get(state_links[9])
time.sleep(3)
def wb_link_route(path):
    LINKS_WB = []
    ROUTE_WB = []
    wait_W = WebDriverWait(driver_W, 10)
    while True:
        try:
            paths = driver_W.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_WB.append(d)
 
            for route in paths:
                ROUTE_WB.append(route.text)
            try:
                active_page_element = driver_W.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_W.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                driver_W.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_W.execute_script("arguments[0].click();", next_page_button)                 
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_WB, ROUTE_WB

# Retrieve links and routes
LINKS_WB, ROUTE_WB = wb_link_route("//a[@class='route']")

# Close the WebDriver
driver_W.quit()

# Save data to a CSV file
df_wb = pd.DataFrame({"Route_name": ROUTE_WB, "Route_link": LINKS_WB})
df_wb.to_csv("west_bengal_route.csv", index=False)

Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found


In [25]:
df_wb

,Route_name,Route_link
0,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
2,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Kolkata to Bolpur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-b...
4,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
5,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
6,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
7,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
8,Midnapore to Barasat (West Bengal),https://www.redbus.in/bus-tickets/midnapore-to...
9,Barasat (West Bengal) to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/barasat-west...


In [26]:
import pandas as pd
df_all=pd.concat([df_k,df_as,df_g,df_h,df_r,df_s,df_t,df_up,df_wb,df_a],ignore_index=True)
df_all.to_csv('total_route_details.csv',index=False)

In [27]:
df_all

,Route_name,Route_link
0,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
...,...,...
347,Visakhapatnam to Rajahmundry,https://www.redbus.in/bus-tickets/visakhapatna...
348,Bangalore to Kurnool,https://www.redbus.in/bus-tickets/bangalore-to...
349,Vijayawada to Kurnool,https://www.redbus.in/bus-tickets/vijayawada-t...
350,Vijayawada to Rajahmundry,https://www.redbus.in/bus-tickets/vijayawada-t...
